## Creating a logistic regression to predict absenteeism

### Import the relevant libraries

In [1]:
import numpy as np
import pandas as pd

In [2]:
data_prepro = pd.read_csv('df_preprocessed.csv')

In [3]:
data_prepro.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700 entries, 0 to 699
Data columns (total 15 columns):
reason_1                     700 non-null int64
reason_2                     700 non-null int64
reason_3                     700 non-null int64
reason_4                     700 non-null int64
Transportation Expense       700 non-null int64
Distance to Work             700 non-null int64
Age                          700 non-null int64
Daily Work Load Average      700 non-null float64
Body Mass Index              700 non-null int64
Education                    700 non-null int64
Children                     700 non-null int64
Pets                         700 non-null int64
Absenteeism Time in Hours    700 non-null int64
Month                        700 non-null int64
Day                          700 non-null int64
dtypes: float64(1), int64(14)
memory usage: 82.1 KB


### Create the Targets

In [4]:
# targets will be the absenteeism hours
# classified as excessive (1) if it is more than the median value, 0 if less
data_prepro['excess absent'] = np.where(data_prepro['Absenteeism Time in Hours'] >
                                             data_prepro['Absenteeism Time in Hours'].median(), 1, 0)

# by using the median we have implicitly balance the dataset

In [5]:
data_prepro = data_prepro.drop(['Absenteeism Time in Hours'], axis=1)

In [6]:
data_prepro['excess absent'].unique()

array([1, 0], dtype=int64)

In [7]:
data_prepro['excess absent'].sum() / len(data_prepro['excess absent'])

# this means around 45% data is one, and around 55% is zero
# usually 60-40 split will work ok in logistic regression
# but not in neural networks
# maybe around 45-55 is the limit for any classification method

0.45571428571428574

In [8]:
targets = np.array(data_prepro['excess absent'])

In [9]:
data_with_targets = data_prepro.copy()

In [10]:
data_with_targets.head()

,reason_1,reason_2,reason_3,reason_4,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Month,Day,excess absent
0,0,0,0,1,289,36,33,239.554,30,0,2,1,7,1,1
1,0,0,0,0,118,13,50,239.554,31,0,1,0,7,1,0
2,0,0,0,1,179,51,38,239.554,31,0,0,0,7,2,0
3,1,0,0,0,279,5,39,239.554,24,0,2,0,7,3,1
4,0,0,0,1,289,36,33,239.554,30,0,2,1,7,3,0


In [11]:
data_with_targets is data_prepro

False

### Select the inputs for the regression

In [12]:
data_with_targets.shape

(700, 15)

In [13]:
data_with_targets.iloc[:,:-1]

,reason_1,reason_2,reason_3,reason_4,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Month,Day
0,0,0,0,1,289,36,33,239.554,30,0,2,1,7,1
1,0,0,0,0,118,13,50,239.554,31,0,1,0,7,1
2,0,0,0,1,179,51,38,239.554,31,0,0,0,7,2
3,1,0,0,0,279,5,39,239.554,24,0,2,0,7,3
4,0,0,0,1,289,36,33,239.554,30,0,2,1,7,3
5,0,0,0,1,179,51,38,239.554,31,0,0,0,7,4
6,0,0,0,1,361,52,28,239.554,27,0,1,4,7,4
7,0,0,0,1,260,50,36,239.554,23,0,4,0,7,4
8,0,0,1,0,155,12,34,239.554,25,0,2,0,7,0
9,0,0,0,1,235,11,37,239.554,29,1,1,1,7,0


In [14]:
unscaled_inputs = data_with_targets.iloc[:,:-1]

### Standardize the data

In [15]:
from sklearn.preprocessing import StandardScaler
# subtract to mean then divide to std for each feature

scaler = StandardScaler()

In [ ]:
scaler.fit()

In [17]:
scaled_inputs = scaler.transform(unscaled_inputs)
scaled_inputs

C:\Users\Public\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  """Entry point for launching an IPython kernel.


array([[-0.57735027, -0.09298136, -0.31448545, ...,  0.26848661,
         0.18272635, -0.68370352],
       [-0.57735027, -0.09298136, -0.31448545, ..., -0.58968976,
         0.18272635, -0.68370352],
       [-0.57735027, -0.09298136, -0.31448545, ..., -0.58968976,
         0.18272635, -0.00772546],
       ...,
       [ 1.73205081, -0.09298136, -0.31448545, ..., -0.58968976,
        -0.3882935 ,  0.66825259],
       [-0.57735027, -0.09298136, -0.31448545, ..., -0.58968976,
        -0.3882935 ,  0.66825259],
       [-0.57735027, -0.09298136, -0.31448545, ...,  0.26848661,
        -0.3882935 ,  0.66825259]])

In [18]:
scaled_inputs.shape

(700, 14)

### Split the data into train & test and shuffle

#### Import the relevant module

In [19]:
from sklearn.model_selection import train_test_split

#### Split

In [20]:
# train_test_split(scaled_inputs, targets, train_size=0.9, shuffle=True, random_state=20)

# split into 4: inputs for training and test, then targets for training and test
# 0.9 or 90% data will be the train data
# default shuffle value is True -> rerun may differ the data also the accuracy!!
# random_state give the option so that the shuffle proceed in the same 'random' way

In [21]:
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs, targets, train_size = 0.8, random_state=20)

C:\Users\Public\Anaconda2\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [22]:
print(x_train.shape, y_train.shape)  # 80%

(560, 14) (560,)


In [23]:
print(x_test.shape, y_test.shape)    # 20%

(140, 14) (140,)


### Logistic regression with sklearn

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

### Training the model

In [25]:
reg = LogisticRegression()

In [26]:
reg.fit(x_train, y_train)

C:\Users\Public\Anaconda2\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [27]:
reg.score(x_train, y_train)

0.7839285714285714

### Manually check the accuracy

In [28]:
# accuracy means that x% of the model output match the targets

model_outputs = reg.predict(x_train)
model_outputs

array([0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0,

In [29]:
y_train

array([0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0,

In [30]:
np.sum(model_outputs==y_train)/model_outputs.shape[0]

# same as reg.score() function above

0.7839285714285714

### Finding the intercept and coefficients

In [31]:
reg.intercept_

array([-0.21957576])

In [32]:
reg.coef_

array([[ 2.07452213,  0.33466331,  1.56097327,  1.32750188,  0.70649077,
        -0.03995965, -0.2009517 , -0.00462411,  0.31937669, -0.13558227,
         0.38168134, -0.33338757,  0.1878226 , -0.07060017]])

In [33]:
unscaled_inputs.columns.values

array(['reason_1', 'reason_2', 'reason_3', 'reason_4',
       'Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Month', 'Day'], dtype=object)

In [34]:
summary_table = pd.DataFrame(data = reg.coef_.T, index = unscaled_inputs.columns.values, columns=['Feature coef'])
summary_table

,Feature coef
reason_1,2.074522
reason_2,0.334663
reason_3,1.560973
reason_4,1.327502
Transportation Expense,0.706491
Distance to Work,-0.039960
Age,-0.200952
Daily Work Load Average,-0.004624
Body Mass Index,0.319377
Education,-0.135582


In [35]:
summary_table.loc['Intercept'] = reg.intercept_[0]
summary_table

,Feature coef
reason_1,2.074522
reason_2,0.334663
reason_3,1.560973
reason_4,1.327502
Transportation Expense,0.706491
Distance to Work,-0.039960
Age,-0.200952
Daily Work Load Average,-0.004624
Body Mass Index,0.319377
Education,-0.135582


### Interpreting the coefficient

In [37]:
summary_table['Odds_ratio'] = np.exp(summary_table)

In [39]:
summary_table.sort_values('Odds_ratio', ascending=False)

# a feature is not particularly important if
# - its coefficient is around 0: no matter the feature value will be multiplied by 0
# - and if its odds ratio is around 1: for a unit change in standardize feature ,
#     the odds increase by to the multiple of odds ratio, 1 means no change
# consider to drop them

,Feature coef,Odds_ratio
reason_1,2.074522,7.960741
reason_3,1.560973,4.763455
reason_4,1.327502,3.771610
Transportation Expense,0.706491,2.026866
Children,0.381681,1.464745
reason_2,0.334663,1.397470
Body Mass Index,0.319377,1.376270
Month,0.187823,1.206619
Daily Work Load Average,-0.004624,0.995387
Distance to Work,-0.039960,0.960828


In [ ]:
# standardizing the dummy variable will lose its interpetabilty of a dummy
# the model is still valid, but we dont know how the different reason compared
# since the values has been standardized

### Standardize the data (again)

In [41]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

In [48]:
class CustomScaler(BaseEstimator, TransformerMixin):
    
    def __init__(self,columns,copy=True,with_mean=True,with_std=True):
        self.scaler = StandardScaler(copy,with_mean,with_std)
        self.columns = columns
        self.mean_ = None
        self.var_ = None
        
    def fit(self,X,y=None):
        self.scaler.fit(X[self.columns],y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self
    
    def transform(self,X,y=None,copy=None):
        init_col_order = X.columns
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]),columns=self.columns)
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        return pd.concat([X_not_scaled,X_scaled], axis=1)[init_col_order]

In [49]:
unscaled_inputs.columns.values

array(['reason_1', 'reason_2', 'reason_3', 'reason_4',
       'Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Month', 'Day'], dtype=object)

In [50]:
columns_to_scale = ['Transportation Expense', 'Distance to Work', 'Age',
                    'Daily Work Load Average', 'Body Mass Index', 'Education',
                    'Children', 'Pets', 'Month', 'Day']

In [51]:
scaler = CustomScaler(columns_to_scale)

In [52]:
scaler.fit(unscaled_inputs)

C:\Users\Public\Anaconda2\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)


CustomScaler(columns=['Transportation Expense', 'Distance to Work', 'Age', 'Daily Work Load Average', 'Body Mass Index', 'Education', 'Children', 'Pets', 'Month', 'Day'],
       copy=None, with_mean=None, with_std=None)

In [53]:
scaled_inputs = scaler.transform(unscaled_inputs)
scaled_inputs

C:\Users\Public\Anaconda2\lib\site-packages\ipykernel_launcher.py:17: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.


,reason_1,reason_2,reason_3,reason_4,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Month,Day
0,0,0,0,1,1.005844,0.412816,-0.536062,-0.806331,0.767431,-0.447980,0.880469,0.268487,0.182726,-0.683704
1,0,0,0,0,-1.574681,-1.141882,2.130803,-0.806331,1.002633,-0.447980,-0.019280,-0.589690,0.182726,-0.683704
2,0,0,0,1,-0.654143,1.426749,0.248310,-0.806331,1.002633,-0.447980,-0.919030,-0.589690,0.182726,-0.007725
3,1,0,0,0,0.854936,-1.682647,0.405184,-0.806331,-0.643782,-0.447980,0.880469,-0.589690,0.182726,0.668253
4,0,0,0,1,1.005844,0.412816,-0.536062,-0.806331,0.767431,-0.447980,0.880469,0.268487,0.182726,0.668253
5,0,0,0,1,-0.654143,1.426749,0.248310,-0.806331,1.002633,-0.447980,-0.919030,-0.589690,0.182726,1.344231
6,0,0,0,1,2.092381,1.494345,-1.320435,-0.806331,0.061825,-0.447980,-0.019280,2.843016,0.182726,1.344231
7,0,0,0,1,0.568211,1.359154,-0.065439,-0.806331,-0.878984,-0.447980,2.679969,-0.589690,0.182726,1.344231
8,0,0,1,0,-1.016322,-1.209478,-0.379188,-0.806331,-0.408580,-0.447980,0.880469,-0.589690,0.182726,-1.359682
9,0,0,0,1,0.190942,-1.277074,0.091435,-0.806331,0.532229,2.232242,-0.019280,0.268487,0.182726,-1.359682


In [54]:
scaled_inputs.shape

(700, 14)

### Split the data into train & test and shuffle

#### Import the relevant module

In [55]:
from sklearn.model_selection import train_test_split

#### Split

In [56]:
# train_test_split(scaled_inputs, targets, train_size=0.9, shuffle=True, random_state=20)

# split into 4: inputs for training and test, then targets for training and test
# 0.9 or 90% data will be the train data
# default shuffle value is True -> rerun may differ the data also the accuracy!!
# random_state give the option so that the shuffle proceed in the same 'random' way

In [57]:
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs, targets, train_size = 0.8, random_state=20)

C:\Users\Public\Anaconda2\lib\site-packages\sklearn\model_selection\_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [58]:
print(x_train.shape, y_train.shape)  # 80%

(560, 14) (560,)


In [59]:
print(x_test.shape, y_test.shape)    # 20%

(140, 14) (140,)


### Logistic regression with sklearn

In [60]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

### Training the model

In [61]:
reg = LogisticRegression()

In [62]:
reg.fit(x_train, y_train)

C:\Users\Public\Anaconda2\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [63]:
reg.score(x_train, y_train)

0.7696428571428572

### Manually check the accuracy

In [64]:
# accuracy means that x% of the model output match the targets

model_outputs = reg.predict(x_train)
model_outputs

array([0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0,

In [65]:
y_train

array([0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0,

In [66]:
np.sum(model_outputs==y_train)/model_outputs.shape[0]

# same as reg.score() function above

0.7696428571428572

### Finding the intercept and coefficients

In [67]:
reg.intercept_

array([-1.50554498])

In [68]:
reg.coef_

array([[ 2.62219702,  0.8411862 ,  2.93554669,  0.67407488,  0.6066182 ,
        -0.00690819, -0.16852671, -0.00369014,  0.26796623, -0.08445828,
         0.35597566, -0.28543538,  0.16153991, -0.08266966]])

In [69]:
unscaled_inputs.columns.values

array(['reason_1', 'reason_2', 'reason_3', 'reason_4',
       'Transportation Expense', 'Distance to Work', 'Age',
       'Daily Work Load Average', 'Body Mass Index', 'Education',
       'Children', 'Pets', 'Month', 'Day'], dtype=object)

In [70]:
summary_table = pd.DataFrame(data = reg.coef_.T, index = unscaled_inputs.columns.values, columns=['Feature coef'])
summary_table

,Feature coef
reason_1,2.622197
reason_2,0.841186
reason_3,2.935547
reason_4,0.674075
Transportation Expense,0.606618
Distance to Work,-0.006908
Age,-0.168527
Daily Work Load Average,-0.003690
Body Mass Index,0.267966
Education,-0.084458


In [71]:
summary_table.loc['Intercept'] = reg.intercept_[0]
summary_table

,Feature coef
reason_1,2.622197
reason_2,0.841186
reason_3,2.935547
reason_4,0.674075
Transportation Expense,0.606618
Distance to Work,-0.006908
Age,-0.168527
Daily Work Load Average,-0.003690
Body Mass Index,0.267966
Education,-0.084458


### Interpreting the coefficient

In [72]:
summary_table['Odds_ratio'] = np.exp(summary_table)

In [73]:
summary_table.sort_values('Odds_ratio', ascending=False)

# a feature is not particularly important if
# - its coefficient is around 0: no matter the feature value will be multiplied by 0
# - and if its odds ratio is around 1: for a unit change in standardize feature ,
#     the odds increase by to the multiple of odds ratio, 1 means no change
# consider to drop them

,Feature coef,Odds_ratio
reason_3,2.935547,18.831795
reason_1,2.622197,13.765934
reason_2,0.841186,2.319116
reason_4,0.674075,1.962217
Transportation Expense,0.606618,1.834218
Children,0.355976,1.427573
Body Mass Index,0.267966,1.307303
Month,0.161540,1.175319
Daily Work Load Average,-0.003690,0.996317
Distance to Work,-0.006908,0.993116


### Backward elimination
The idea is that we can simplify the model by removing all features which have close to no contribution to the model.

When we have the p-values, we get rid of all coeff with p-value > 0.05.

If the weight is small enough , it wont make a difference anyway. If we remove these variables, the rest of the model should not really change in terms of coefficient values.

In [ ]:
# lets drop 'Day', 'Daily Work Load Average', 'Distance to Work'